# Support Vector Machines - Salary Dataset

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
train = pd.read_csv("C:/Users/BASHAR/OneDrive/Documents/Data Science/Assignment/Assignment - 17/SalaryData_Train(1).csv")
test = pd.read_csv("C:/Users/BASHAR/OneDrive/Documents/Data Science/Assignment/Assignment - 17/SalaryData_Test(1).csv")

In [3]:
train.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
test.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,34,Private,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


## EDA on Train data

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [6]:
train.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [7]:
train[train.duplicated()].shape

(3258, 14)

In [8]:
train[train.duplicated()]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
326,33,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
364,27,Private,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,0,0,50,United-States,<=50K
518,24,Private,HS-grad,9,Never-married,Handlers-cleaners,Unmarried,Black,Female,0,0,40,United-States,<=50K
525,24,Private,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
619,33,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30110,47,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
30118,20,Private,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,40,United-States,<=50K
30132,22,Private,Some-college,10,Never-married,Adm-clerical,Own-child,White,Male,0,0,40,United-States,<=50K
30157,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K


In [9]:
train1 = train.drop_duplicates()
train1.iloc[325:330]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
325,26,Private,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,<=50K
327,46,Local-gov,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,30,United-States,<=50K
328,38,Federal-gov,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,>50K
329,65,Private,11th,7,Widowed,Other-service,Unmarried,Other,Male,0,0,40,United-States,<=50K
330,40,Private,Assoc-voc,11,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,>50K


In [10]:
train2 = train1.reset_index()
train2 = train2.drop(columns="index")
train2.iloc[325:330]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
325,26,Private,HS-grad,9,Married-civ-spouse,Other-service,Husband,White,Male,0,0,40,United-States,<=50K
326,46,Local-gov,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,30,United-States,<=50K
327,38,Federal-gov,HS-grad,9,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States,>50K
328,65,Private,11th,7,Widowed,Other-service,Unmarried,Other,Male,0,0,40,United-States,<=50K
329,40,Private,Assoc-voc,11,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States,>50K


In [11]:
labelencoder = LabelEncoder()

In [12]:
names = train2[["workclass", "education", "maritalstatus", "occupation", "relationship", "race", "sex", "native", "Salary"]]

In [13]:
for i in names:
    train2[i] = labelencoder.fit_transform(train2[i])
    
train2.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,39,5,9,13,4,0,1,4,1,2174,0,40,37,0
1,50,4,9,13,2,3,0,4,1,0,0,13,37,0
2,38,2,11,9,0,5,1,4,1,0,0,40,37,0
3,53,2,1,7,2,5,0,2,1,0,0,40,37,0
4,28,2,9,13,2,9,5,2,0,0,0,40,4,0


## EDA on Test data

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15060 entries, 0 to 15059
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            15060 non-null  int64 
 1   workclass      15060 non-null  object
 2   education      15060 non-null  object
 3   educationno    15060 non-null  int64 
 4   maritalstatus  15060 non-null  object
 5   occupation     15060 non-null  object
 6   relationship   15060 non-null  object
 7   race           15060 non-null  object
 8   sex            15060 non-null  object
 9   capitalgain    15060 non-null  int64 
 10  capitalloss    15060 non-null  int64 
 11  hoursperweek   15060 non-null  int64 
 12  native         15060 non-null  object
 13  Salary         15060 non-null  object
dtypes: int64(5), object(9)
memory usage: 1.6+ MB


In [15]:
test.isna().sum()

age              0
workclass        0
education        0
educationno      0
maritalstatus    0
occupation       0
relationship     0
race             0
sex              0
capitalgain      0
capitalloss      0
hoursperweek     0
native           0
Salary           0
dtype: int64

In [16]:
test[test.duplicated()].shape

(930, 14)

In [17]:
test[test.duplicated()]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
551,59,Private,HS-grad,9,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
601,32,Private,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States,<=50K
681,40,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
799,24,Private,Bachelors,13,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,35,United-States,<=50K
1044,44,Private,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,60,United-States,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,51,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
15044,22,Private,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
15049,32,Private,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States,<=50K
15052,31,Private,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K


In [18]:
test1 = test.drop_duplicates()
test1.iloc[550:555]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
550,49,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
552,38,Private,HS-grad,9,Separated,Machine-op-inspct,Unmarried,Black,Female,0,0,28,United-States,<=50K
553,50,Self-emp-inc,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,55,United-States,<=50K
554,51,Self-emp-inc,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1672,55,United-States,<=50K
555,22,Private,Some-college,10,Never-married,Machine-op-inspct,Own-child,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K


In [19]:
test2 = test1.reset_index()
test2 = test2.drop(columns="index")
test2.iloc[550:555]

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
550,49,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K
551,38,Private,HS-grad,9,Separated,Machine-op-inspct,Unmarried,Black,Female,0,0,28,United-States,<=50K
552,50,Self-emp-inc,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,55,United-States,<=50K
553,51,Self-emp-inc,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,1672,55,United-States,<=50K
554,22,Private,Some-college,10,Never-married,Machine-op-inspct,Own-child,Asian-Pac-Islander,Male,0,0,35,United-States,<=50K


In [20]:
for i in names:
    test2[i] = labelencoder.fit_transform(test2[i])
    
test2.head()

,age,workclass,education,educationno,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native,Salary
0,25,2,1,7,4,6,3,2,1,0,0,40,37,0
1,38,2,11,9,2,4,0,4,1,0,0,50,37,0
2,28,1,7,12,2,10,0,4,1,0,0,40,37,1
3,44,2,15,10,2,6,0,2,1,7688,0,40,37,1
4,34,2,0,6,4,7,1,4,1,0,0,30,37,0


## Train & Test Split

In [21]:
x_train = train2.iloc[:, 0:13]
y_train = train2.iloc[:, 13]
x_test = test2.iloc[:, 0:13]
y_test = test2.iloc[:, 13]

## Normalization

In [22]:
def norm_func (i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [23]:
x_train = norm_func(x_train)
x_test =  norm_func(x_test)

## Building model using Support Vector Machines

### kernel = Linear

In [28]:
model_linear = SVC(kernel='linear', random_state=7, gamma=100, C=15)
model_linear.fit(x_train, y_train)

SVC(C=15, gamma=100, kernel='linear', random_state=7)

In [29]:
pred_test_linear = model_linear.predict(x_test)
pred_test_linear

array([0, 0, 0, ..., 0, 1, 0])

In [30]:
confusion_matrix(y_test, pred_test_linear)

array([[10250,   370],
       [ 2417,  1093]], dtype=int64)

In [31]:
accuracy = accuracy_score(y_test, pred_test_linear)
print((accuracy)*100)

80.276008492569


In [32]:
print("The accuracy of the svm classifier on training data is {:.2f} out of 1".format(model_linear.score(x_train, y_train)*100))

print("The accuracy of the svm classifier on test data is {:.2f} out of 1".format(model_linear.score(x_test, y_test)*100))

The accuracy of the svm classifier on training data is 80.55 out of 1
The accuracy of the svm classifier on test data is 80.28 out of 1


### kernel = RBF

In [33]:
model_rbf = SVC(kernel = "rbf", random_state=7, gamma=100, C=15)
model_rbf.fit(x_train,y_train)

SVC(C=15, gamma=100, random_state=7)

In [34]:
pred_test_rbf = model_rbf.predict(x_test)
pred_test_rbf

array([0, 0, 1, ..., 1, 0, 1])

In [35]:
confusion_matrix(y_test, pred_test_rbf)

array([[9698,  922],
       [2017, 1493]], dtype=int64)

In [36]:
accuracy = accuracy_score(y_test, pred_test_rbf)
print((accuracy)*100)

79.2002830856334


In [37]:
print("The accuracy of the svm classifier on training data is {:.2f} out of 1".format(model_rbf.score(x_train, y_train)*100))

print("The accuracy of the svm classifier on test data is {:.2f} out of 1".format(model_rbf.score(x_test, y_test)*100))

The accuracy of the svm classifier on training data is 95.30 out of 1
The accuracy of the svm classifier on test data is 79.20 out of 1


### Conclusion:
    
Linear Model gives the best accuracy.